In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-382202-fa58f9c946e5.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/08"
pmc = "08003010" # 店舗ID
SPREADSHEET_KEY = '1Lf-urEfRxYOqbkF_Hu9o62GyZ_n-9x5Q9PIp80LCAec' # スプレッドシートのID
#pstart = 1 # パチンコの1台目
sstart = 1 # スロットの1台目
wait = 0.02

user_id = "g84909252+002@gmail.com"
user_pw = "sitega21"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

s20table = []
s5table = []
s2table = []



# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "555":
                    s5table.append(li)
                if unit == "222":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "555":
                        s5table.append(li)
                    if unit == "222":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['08/08', '120110', '1', '2739', '3565.0', '31', '14', '--']
['08/08', '120110', '2', '2755', '370.0', '18', '10', '--']
['08/08', '120110', '3', '4974', '-4583.0', '13', '4', '--']
['08/08', '120110', '5', '2656', '-833.0', '13', '6', '--']
['08/08', '120110', '6', '1266', '1852.0', '16', '6', '--']
['08/08', '120110', '7', '7063', '-1991.0', '36', '17', '--']
['08/08', '120110', '8', '1886', '3843.0', '25', '14', '--']
['08/08', '120110', '10', '2918', '-1759.0', '10', '6', '--']
['08/08', '120110', '11', '2694', '4352.0', '34', '14', '--']
['08/08', '120110', '12', '2559', '741.0', '21', '5', '--']
['08/08', '120110', '13', '551', '602.0', '5', '4', '--']
['08/08', '120110', '15', '1741', '4259.0', '29', '8', '--']
['08/08', '120110', '16', '3538', '-2222.0', '12', '6', '--']
['08/08', '120110', '17', '1565', '-926.0', '6', '4', '--']
['08/08', '120110', '18', '2499', '-648.0', '12', '7', '--']
['08/08', '120110', '20', '813', '-417.0', '3', '2', '--']
['08/08', '120110', '21', '337

['08/08', '120073', '211', '989', '370.0', '0', '3', '10']
['08/08', '120073', '212', '1610', '1620.0', '0', '3', '18']
['08/08', '120073', '213', '955', '1343.0', '0', '1', '13']
['08/08', '120073', '215', '990', '-93.0', '0', '4', '7']
['08/08', '120073', '216', '1483', '-1574.0', '0', '2', '3']
['08/08', '120073', '217', '2223', '-324.0', '0', '8', '15']
['08/08', '120073', '218', '2162', '1574.0', '0', '6', '21']
['08/08', '120073', '220', '1264', '-880.0', '0', '5', '5']
['08/08', '120073', '221', '1046', '-417.0', '0', '3', '6']
['08/08', '120073', '222', '4398', '972.0', '0', '12', '35']
['08/08', '120073', '223', '1670', '694.0', '0', '5', '15']
['08/08', '120121', '225', '1139', '-648.0', '0', '1', '13']
['08/08', '120121', '226', '1059', '-1250.0', '0', '1', '3']
['08/08', '120077', '227', '2664', '-972.0', '0', '4', '18']
['08/08', '120077', '228', '3601', '3519.0', '0', '22', '42']
['08/08', '120077', '230', '3124', '-880.0', '0', '7', '24']
['08/08', '119897', '231', '0', 

['08/08', '120129', '528', '1138', '-185.0', '0', '3', '7']
['08/08', '120145', '530', '6222', '-787.0', '0', '12', '54']
['08/08', '120145', '531', '759', '-741.0', '0', '1', '3']
['08/08', '120145', '532', '4328', '1898.0', '0', '7', '52']
['08/08', '120145', '533', '3624', '-1111.0', '0', '5', '26']
['08/08', '120145', '535', '2885', '-278.0', '0', '4', '25']
['08/08', '120142', '536', '5680', '-4167.0', '0', '19', '26']
['08/08', '120142', '537', '6148', '-694.0', '0', '18', '58']
['08/08', '120138', '538', '486', '-417.0', '0', '1', '1']
['08/08', '120138', '550', '3513', '1343.0', '0', '7', '14']
['08/08', '120128', '551', '1163', '-1343.0', '0', '0', '1']
['08/08', '120128', '552', '0', '0', '0', '0', '--']
['08/08', '120128', '553', '333', '-93.0', '0', '0', '1']
['08/08', '120128', '555', '317', '-417.0', '0', '0', '--']
['08/08', '120128', '556', '1296', '-139.0', '0', '0', '4']
['08/08', '120104', '557', '414', '-231.0', '0', '0', '2']
['08/08', '120104', '558', '1504', '-16

{'spreadsheetId': '1Lf-urEfRxYOqbkF_Hu9o62GyZ_n-9x5Q9PIp80LCAec',
 'updates': {'spreadsheetId': '1Lf-urEfRxYOqbkF_Hu9o62GyZ_n-9x5Q9PIp80LCAec',
  'updatedRange': 'S2RAW!F1234:M1289',
  'updatedRows': 56,
  'updatedColumns': 8,
  'updatedCells': 448}}

In [ ]:
#date = 0 #当日は0、前日は1・・・
#
date = 1 #当日は0、前日は1・・・
date2 = "07/13"
pmc = "08003010" # 店舗ID
SPREADSHEET_KEY = '1Lf-urEfRxYOqbkF_Hu9o62GyZ_n-9x5Q9PIp80LCAec' # スプレッドシートのID
#pstart = 1 # パチンコの1台目
sstart = 1 # スロットの1台目
wait = 0.02

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

s20table = []
s5table = []
s2table = []



# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "555":
                    s5table.append(li)
                if unit == "222":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "555":
                        s5table.append(li)
                    if unit == "222":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['07/13', '120110', '1', '3485', '-1944.0', '14', '11', '--']
['07/13', '120110', '2', '3891', '-278.0', '21', '17', '--']
['07/13', '120110', '3', '2508', '231.0', '16', '7', '--']
['07/13', '120110', '5', '1809', '463.0', '14', '4', '--']
['07/13', '120110', '6', '240', '-231.0', '0', '0', '--']
['07/13', '120110', '7', '2663', '-324.0', '16', '7', '--']
['07/13', '120110', '8', '1608', '-1157.0', '4', '5', '--']
['07/13', '120110', '10', '2465', '-741.0', '13', '6', '--']
['07/13', '120110', '11', '1459', '-556.0', '7', '1', '--']
['07/13', '120110', '12', '888', '-509.0', '3', '2', '--']
['07/13', '120110', '13', '2245', '-2037.0', '9', '8', '--']
['07/13', '120110', '15', '732', '324.0', '6', '3', '--']
['07/13', '120110', '16', '1952', '-1065.0', '9', '0', '--']
['07/13', '120110', '17', '191', '-231.0', '0', '0', '--']
['07/13', '120110', '18', '525', '4444.0', '21', '8', '--']
['07/13', '120110', '20', '1364', '417.0', '11', '2', '--']
['07/13', '120110', '21', '213', '1065.0',